In [1]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
import sympy
import os
root_folder = os.getcwd()

In [2]:
# v, f = igl.read_triangle_mesh(os.path.join(root_folder, "rbc.off"))

In [3]:
import meshzoo
import optimesh
points,cells = meshzoo.icosa_sphere(11)

class Sphere:
    def f(self, x):
        return 1.0 - (x[0] ** 2 + x[1] ** 2 + x[2] ** 2)

    def grad(self, x):
        return -2 * x
v_init, f= optimesh.optimize_points_cells(
    points,cells,"CVT (full)",1.0e-2,100,verbose=False,implicit_surface=Sphere(),
    # step_filename_format="out{:03d}.vtk"
)
rc=igl.avg_edge_length(v_init,f)*1.72
mp.jupyter()
p=mp.plot(v_init,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)
        #p.add_lines(v,v+1*FB, shading={"line_color": "red"})
print(len(points))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1212


In [4]:
x1=v_init[:,0]
y1=v_init[:,1]
z1=v_init[:,2]
v_init[1]

array([5.25731112e-01, 8.50650808e-01, 6.66790581e-16])

In [5]:
C_0=0.2702
C_1=2.0026
C_2=-1.1228
for i in range(len(x1)):
    rho=np.sqrt( (x1[i])**2 + (y1[i])**2 )
    if z1[i] >=0:
        z1[i]=((np.sqrt(abs((1-rho**2))))/2)*(C_0 +C_1*rho**2+C_2*rho**4)
    else:
        z1[i]=((np.sqrt(abs((1-rho**2))))/2)*(C_0 +C_1*rho**2+C_2*rho**4)*(-1)   

In [23]:
v = np.array([x1, y1,z1]).T

In [24]:
p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [25]:
npv = igl.per_vertex_normals(v, f)
K = igl.gaussian_curvature(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
minv = sp.sparse.diags(1 / m.diagonal())
area_voronoi=m.diagonal()
l = igl.cotmatrix(v, f) ###laplacian-operator
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
minv = sp.sparse.diags(1 / m.diagonal())
Hn = -minv.dot(l.dot(v))/2
H_mean = np.linalg.norm(Hn, axis=1)
sign_H = np.sign(np.sum(Hn*npv, axis=1))
#if (min(sign_H)<0):
#    print('H_mean changes sign')
H_mean_signed = H_mean*sign_H

In [26]:
H_mean_avg=np.average(H_mean_signed)

In [13]:
import sympy
from sympy import *
x,y,z=symbols("x y z")

In [14]:
C_0=0.2072
C_1=2.0026
C_2=-1.228
rho=sqrt(x**2 + y**2)
expression=((sqrt(1-rho**2)/2)*(C_0+(C_1*rho**2)+ C_2*rho**4))**2  - z**2
# simplify(rho)
#expression= (x**2 + y**2 +z**2 -1)
simplify(expression)

-z**2 - 1.00260169*(x**2 + y**2 - 1)*(x**2 + y**2 - 0.613202836312793*(x**2 + y**2)**2 + 0.103465494856686)**2

In [15]:
fx=diff(expression,x)
fy=diff(expression,y)
fz=diff(expression,z)

fxx=diff(fx,x)
fxy=diff(fx,y)
fxz=diff(fx,z)

fyx=diff(fy,x)
fyy=diff(fy,y)
fyz=diff(fy,z)

fzx=diff(fz,x)
fzy=diff(fz,y)
fzz=diff(fz,z)


In [17]:
delf= Matrix([[fx, fy, fz]]) ###gradient of explicit function
delf_T=delf.T
Hessian = Matrix([[fxx, fxy, fxz], [fyx, fyy, fyz],[fzx, fzy, fzz]]) ####Hessian Matrix
Trace_Hessian=trace(Hessian)
del_f_mag=sqrt(delf[0]**2 +delf[1]**2+ delf[2]**2) ###Magnitude of Gradient Matrix
AA=delf*Hessian*delf_T
aa=AA.det()
Mean_Curvature=(aa - (((del_f_mag)**2)*Trace_Hessian))/(2*(del_f_mag)**3)
pprint(Mean_Curvature)

                   ⎛                                                                                                                       
                   ⎜         ⎛                                                                           2                                 
                   ⎜         ⎜               ⎛                                     2                    ⎞                                  
                   ⎜     2   ⎜               ⎜ 2    2                     ⎛ 2    2⎞                     ⎟               ⎛                  
0.0620147104579334⋅⎝- 8⋅z  + ⎝- 2.00520338⋅x⋅⎝x  + y  - 0.613202836312793⋅⎝x  + y ⎠  + 0.103465494856686⎠  + 1.00260169⋅⎝- 4.90562269050235
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                           
                    

In [19]:
Mean_Curvature_Analytical=np.zeros(len(x1))
for i in range(len(x1)):
    Mean_Curv_Values=Mean_Curvature.subs(x,x1[i]).subs(y,y1[i]).subs(z,z1[i])
    Mean_Curvature_Analytical[i]=Mean_Curv_Values.evalf()
    

In [21]:
p=mp.plot(v,f,Mean_Curvature_Analytical,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [28]:
p=mp.plot(v,f,H_mean_signed,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…